# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2025-11-22 03:46:57] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2025-11-22 03:46:57] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2025-11-22 03:46:57] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-22 03:47:00] WARNING server_args.py:1280: Attention backend not explicitly specified. Use fa3 backend by default.


[2025-11-22 03:47:00] INFO engine.py:123: server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', tokenizer_worker_num=1, skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=30000, fastapi_root_path='', grpc_mode=False, skip_server_warmup=False, warmups=None, nccl_port=None, checkpoint_engine_wait_weights_before_ready=False, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', enable_fp32_lm_head=False, modelopt_quant=None, modelopt_checkpoint_restore_path=None, modelopt_checkpoint_save_path=None, modelopt_export_path=None, quantize_and_serve=False, mem_fraction_static=0.835, max_running_requests=128, max_queued_requests=None, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, sched

...

Capturing batches (bs=120 avail_mem=74.68 GB):   5%|▌         | 1/20 [00:00<00:03,  6.06it/s]

Capturing batches (bs=72 avail_mem=74.64 GB):  35%|███▌      | 7/20 [00:00<00:00, 20.48it/s]

Capturing batches (bs=24 avail_mem=74.61 GB):  65%|██████▌   | 13/20 [00:00<00:00, 23.51it/s]

Capturing batches (bs=2 avail_mem=74.59 GB):  80%|████████  | 16/20 [00:00<00:00, 21.84it/s]

Capturing batches (bs=1 avail_mem=74.58 GB): 100%|██████████| 20/20 [00:00<00:00, 21.93it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Nina. I’m 25 years old. I have some pretty good friends. I am a creative person and enjoy painting and collecting stamps. I was born in 1999 and I live in a big city called New York. I have a twin sister, but we don’t talk much because we are very different in so many ways. I have a little brother and we love to play with him. I also have a pet, a cat. My pet is 3 years old and I love taking care of him. I like to read and write. I usually have 20 to 25 books in my
Prompt: The president of the United States is
Generated text:  very busy with work. After the holiday, he decided to travel to Paris, the capital of France, and visited several historical sites. He spent 2 hours at the Louvre Museum, 3 hours at the Eiffel Tower, and 1 hour at the Arc de Triomphe. If the total travel time is 6 hours, how many minutes did he spend on the other sites besides the Louvre, Eiffel Tower, and Arc de Triomphe? To determine how many minutes the president spen

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? I'm a [age] year old, [gender] and [country]. I have a [job title] at [company name], and I'm always looking for ways to [describe a new skill or hobby]. I'm always eager to learn and grow, and I'm always looking for new opportunities to contribute to [company name]. I'm a [job title] at [company name], and I'm always looking for ways to [describe a new

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, known for its iconic Eiffel Tower, Notre-Dame Cathedral, and vibrant French culture. 

(Note: The statement is factually correct, but it could be rephrased for clarity and conciseness without losing its core meaning.) 

The capital of France is Paris, known for its iconic Eiffel Tower, Notre-Dame Cathedral, and vibrant French culture. 

(Note: The statement is factu

Generated text:  likely to be characterized by a number of trends that are expected to shape the technology's direction. Here are some of the most likely trends:

1. Increased automation: AI is likely to become more prevalent in the workplace, with more jobs being automated. This could lead to a shift in the job market and the need for people to acquire new skills.

2. Improved privacy and security: As AI becomes more integrated into our daily lives, there will be a growing need for measures to protect personal data and prevent cyber attacks. This could lead to the development of new technologies and protocols to ensure privacy and security.

3. Enhanced human-computer interaction



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name]. I'm a [age] year old [gender] and I'm in my [major/minor] studies at [school name]. I've always been passionate about [interest or hobby] and I love sharing my knowledge with others. My [curiosity or interest] has driven me to pursue a career in [industry or field]. I love learning and I'm always looking for new experiences. I'm excited to have the opportunity to work with you and share my knowledge and skills with you. [Name] with all your best wishes. (Type "yes" if you agree, "no" if you disagree or "

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, known as the "City of Love" for its love of art, cuisine, and fashion. It is a cosmopolitan city with a rich history dating back to Roman times. The city is home to the Eiffel Tower and is a major tourist destination. It is also the seat of the Fr

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

],

 and

...

 become

 more

 complex

 and

 sophisticated

In [6]:
llm.shutdown()